<a href="https://colab.research.google.com/github/jinxianyap/classification-models/blob/master/metrics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [185]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive/My\ Drive/WISE AI/Classification

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).
/gdrive/My Drive/WISE AI/Classification


In [186]:
from keras.models import model_from_json
import numpy 
import os
import math
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from numpy import asarray
from keras.optimizers import SGD,Adam
from sklearn.model_selection import train_test_split

target_height = 200
target_width = 200

In [187]:
def load_dataset():
  categories = ['blur', 'normal', 'dark', 'glare', 'damaged', 'covered']
  classes = os.listdir('./classes')
  train_photos, test_photos, train_labels, test_labels = list(), list(), list(), list()

  def get_encoding(class_name):
    return list(map(lambda c: 1 if c == class_name else 0, categories))

  for each in classes:
    i = 0;
    for image in os.listdir('./classes/' + each):
      output = get_encoding(each)
      photo = load_img('./classes/' + each + "/" + image, target_size=(target_height, target_width))
      photo = img_to_array(photo)

      if i > 2:
        train_photos.append(photo)
        train_labels.append(output)
      else:
        test_photos.append(photo)
        test_labels.append(output)
      i += 1;

  X_train = asarray(train_photos)
  y_train = asarray(train_labels)
  X_test = asarray(test_photos)
  y_test = asarray(test_labels)

  return X_train, y_train, X_test, y_test

In [188]:
def load_basic_classification():
  basic_class_file = open('./models/basic_classification.json', 'r')
  basic_class_model_json = basic_class_file.read()
  basic_class_file.close()
  basic_class_model = model_from_json(basic_class_model_json)
  basic_class_model.load_weights("./weights/basic_classification.h5")
  print("Loaded basic classification model from disk")
  
  basic_class_model.compile(loss='categorical_crossentropy', optimizer=SGD(lr=0.01, momentum=0.9), metrics=['accuracy'])
  return basic_class_model

In [189]:
def load_multi_label_classification():
  multi_label_file = open('./models/multi_label_classification.json', 'r')
  multi_label_model_json = multi_label_file.read()
  multi_label_file.close()
  multi_label_model = model_from_json(multi_label_model_json)
  multi_label_model.load_weights("./weights/multi_label_classification.h5")
  print("Loaded multi label classification model from disk")
  
  multi_label_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
  return multi_label_model

In [190]:
def evaluate_model(model, X, y):
  avgs = 0
  for i in range(2, 12):
    X_split, X_back, y_split, y_back = train_test_split(X, y, test_size=0.3, shuffle=True, random_state=i)
    score = model.evaluate(X_back, y_back, verbose=0)
    print("%s: %.2f%%" % (model.metrics_names[1], score[1]*100))
    avgs = avgs + score[1]

  print("Average accuracy: %.2f%%" % (avgs / 10 * 100))

In [191]:
X_train, y_train, X_test, y_test = load_dataset()

# basic
basic_model = load_basic_classification()
evaluate_model(basic_model, X_test, y_test)

# multi label
multi_label_model = load_multi_label_classification()
evaluate_model(multi_label_model, X_test, y_test)


Loaded basic classification model from disk
accuracy: 83.33%
accuracy: 83.33%
accuracy: 83.33%
accuracy: 83.33%
accuracy: 83.33%
accuracy: 66.67%
accuracy: 83.33%
accuracy: 66.67%
accuracy: 100.00%
accuracy: 83.33%
Average accuracy: 81.67%
Loaded multi label classification model from disk
accuracy: 83.33%
accuracy: 50.00%
accuracy: 50.00%
accuracy: 83.33%
accuracy: 83.33%
accuracy: 83.33%
accuracy: 66.67%
accuracy: 50.00%
accuracy: 83.33%
accuracy: 83.33%
Average accuracy: 71.67%
